In [1]:
from MesoPy import Meso
import numpy as np
import os
import datetime
import sys

In [3]:
#place inside ~/.bashrc
#export mesokey='key'
m=Meso(token=os.environ['mesokey'])

In [136]:
#Goal is to limit to just one observation a day. Air temp high and low we can likely just take from a specific slot
#perhaps the first

import pandas as pd
df_snow = pd.read_csv('../data/CAICVailSummitDangerScale.csv')
df_snow=df_snow.drop_duplicates(subset='date')
df_snow = df_snow.set_index('date')
#df_weather = pd.read_csv('weather.csv')
df_weather_date = pd.read_csv('weather_date.csv')
df_weather_date = df_weather_date.drop_duplicates()
df_weather_date = df_snow.join(df_weather_date.set_index('date'))
df_weather_date = df_weather_date.set_index('datetime')

In [150]:
columns = ['datetime', 
           'yesterday_air_temp_low_24_hour',
           'yesterday_air_temp_high_24_hour', 
           'yesterday_precip_accum_24_hour',
           'yesterday_rating_above',
           'yesterday_rating_near',
           'yesterday_rating_below']
df_y = pd.DataFrame(data=np.zeros((0,len(columns))), columns=columns)
for index, row in df_weather_date.iterrows():
    y_date = pd.to_datetime(index)-pd.DateOffset(1)
    y_date = y_date.date().strftime('%Y-%m-%d')
    #help(y_date)
    #print(y_date.date(), index)
    #break
    try:
        y=df_weather_date.loc[y_date]
        if(y.shape==(2,7)):
            print(y_date)
        df_y = df_y.append({'datetime': index, 
                            'yesterday_air_temp_low_24_hour':y['air_temp_low_24_hour'], 
                            'yesterday_air_temp_high_24_hour': y['air_temp_high_24_hour'], 
                            'yesterday_precip_accum_24_hour':y['precip_accum_24_hour'],
                            'yesterday_rating_above': y['rating_today_above'],
                            'yesterday_rating_near': y['rating_today_near'],
                            'yesterday_rating_below': y['rating_today_below']}, ignore_index=True)
    except:
        print("not found", y_date)

not found 2013-12-22
not found 2014-06-29
not found 2014-11-18
not found 2015-04-21
not found 2015-05-06
not found 2015-11-01
not found 2015-11-12
not found 2016-04-24
not found 2016-05-25
not found 2016-10-12
not found 2016-10-18
not found 2016-11-21


In [151]:
df_run = df_weather_date.join(df_y.set_index('datetime'))
df_clean = df_run.dropna(axis=0, how='any')
df=df_clean

In [172]:
from sklearn.cross_validation import train_test_split
X, Y = df.iloc[:,3:].values, df.iloc[:, 0].values
X_train, X_test, Y_train, Y_test = \
    train_test_split(X, Y, test_size=0.3, random_state=0)

In [168]:
import sklearn.preprocessing as pp
#help(pp)
stdsc = pp.StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_test)


In [169]:
from sklearn.linear_model import LogisticRegression
LogisticRegression(penalty='l1')
lr=LogisticRegression(penalty='l1', C=0.1)
lr.fit(X_train_std, Y_train)
print('train accuracy: ', lr.score(X_train_std, Y_train))
print('test accuracy: ', lr.score(X_test_std, Y_test))

train accuracy:  0.776836158192
test accuracy:  0.756578947368


/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [174]:
from sklearn import neighbors
clf = neighbors.KNeighborsClassifier(15,weights='uniform')
clf.fit(X_train, Y_train)
print('train accuracy: ', clf.score(X_train, Y_train))
print('test accuracy: ', clf.score(X_test, Y_test))

train accuracy:  0.703389830508
test accuracy:  0.565789473684


In [ ]:
#try decision tress/forests next